In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [51]:
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm

In [143]:
data  = pd.read_csv('problem_2_train.csv',parse_dates=['memcreateddate', 'mobilepurchasedate','claimintdate','pickupStartdate'])
data.head()

,srno,mobmake,hubid,hubpincode,pickuppartnerid,deliverypartnerid,insuranceCompanyId,custpincodecategory,claimtype,custpincode,memcreateddate,mobilepurchasedate,claimintdate,servicecntrid,pickupStartdate,tat_in_days
0,2851218,AN,7,500004,233,233,131,A,ADLD,500001,2017-04-27,2016-12-02,2018-02-08,4403,2018-02-13,5
1,2838330,AJ,10,226010,233,233,131,A,ADLD,226022,2017-03-06,2017-03-05,2018-02-04,4444,2018-02-08,12
2,2835781,AJ,2,110018,233,233,228,A,ADLD,110018,2018-02-01,2018-01-27,2018-02-03,4388,2018-02-12,9
3,2838589,AM,2,110018,233,233,228,B,ADLD,124001,2018-01-17,2018-01-31,2018-02-04,4092,2018-02-14,9
4,2855214,AN,21,110015,233,233,131,A,ADLD,201301,2018-01-23,2018-01-20,2018-02-09,4580,2018-02-14,8


In [144]:
data.insuranceCompanyId.value_counts()

131    4701
228    1549
Name: insuranceCompanyId, dtype: int64

### Making new features as diffrerence of dates

In [145]:
data['mem-purchase'] = (data.mobilepurchasedate - data.memcreateddate).dt.days
data['pickup-claim'] = (data.pickupStartdate - data.claimintdate).dt.days
data['claim-mem'] = (data.claimintdate - data.memcreateddate).dt.days
data['claim-purchase'] = (data.claimintdate - data.mobilepurchasedate).dt.days

### Making maps for categorical data....
### Alternative way for Label encoding

In [146]:
claim = list(data.claimtype.unique())
mob = list(data.mobmake.unique())
pin_cat  = list(data.custpincodecategory.unique())
ins = list(data.insuranceCompanyId.unique())


claim_map = {k:v for v,k in enumerate(claim)}
mob_map = {k:v for v,k in enumerate(mob)}
pin_map = {k:v for v,k in enumerate(pin_cat)}
ins_map = {k:v for v,k in enumerate(ins)}

print(ins_map)

{131: 0, 228: 1}


In [147]:

data.claimtype = data.claimtype.map(claim_map)
data.mobmake = data.mobmake.map(mob_map)
data.custpincodecategory = data.custpincodecategory.map(pin_map)
data.insuranceCompanyId = data.insuranceCompanyId.map(ins_map)

data.head()

,srno,mobmake,hubid,hubpincode,pickuppartnerid,deliverypartnerid,insuranceCompanyId,custpincodecategory,claimtype,custpincode,memcreateddate,mobilepurchasedate,claimintdate,servicecntrid,pickupStartdate,tat_in_days,mem-purchase,pickup-claim,claim-mem,claim-purchase
0,2851218,0,7,500004,233,233,0,0,0,500001,2017-04-27,2016-12-02,2018-02-08,4403,2018-02-13,5,-146,5,287,433
1,2838330,1,10,226010,233,233,0,0,0,226022,2017-03-06,2017-03-05,2018-02-04,4444,2018-02-08,12,-1,4,335,336
2,2835781,1,2,110018,233,233,1,0,0,110018,2018-02-01,2018-01-27,2018-02-03,4388,2018-02-12,9,-5,9,2,7
3,2838589,2,2,110018,233,233,1,1,0,124001,2018-01-17,2018-01-31,2018-02-04,4092,2018-02-14,9,14,10,18,4
4,2855214,0,21,110015,233,233,0,0,0,201301,2018-01-23,2018-01-20,2018-02-09,4580,2018-02-14,8,-3,5,17,20


In [148]:
data.columns

Index(['srno', 'mobmake', 'hubid', 'hubpincode', 'pickuppartnerid',
       'deliverypartnerid', 'insuranceCompanyId', 'custpincodecategory',
       'claimtype', 'custpincode', 'memcreateddate', 'mobilepurchasedate',
       'claimintdate', 'servicecntrid', 'pickupStartdate', 'tat_in_days',
       'mem-purchase', 'pickup-claim', 'claim-mem', 'claim-purchase'],
      dtype='object')

In [164]:
X = data[['mobmake', 'hubid', 'hubpincode', 'insuranceCompanyId', 'custpincodecategory',
       'claimtype', 'custpincode','servicecntrid','mem-purchase', 'pickup-claim','claim-mem','claim-purchase']]
y = data.tat_in_days

In [165]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3)

### Linear Regression

In [168]:
lr = LinearRegression()
lr.fit(X_train,y_train)
lr_pred = lr.predict(X_test)
lr_mse = metrics.mean_squared_error(y_test,lr_pred)

print('Training Score:',lr.score(X_train, y_train))
print('Testing Score:',lr.score(X_test, y_test))
print('Root Mean squared Error:',np.sqrt(lr_mse))

Training Score: 0.1792546819545604
Testing Score: 0.17341318038615902
Root Mean squared Error: 4.712435643701893


### Random Forrest Regressor

In [210]:
rf = RandomForestRegressor(criterion='mse')
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
rf_mse = metrics.mean_squared_error(y_test,rf_pred)

print('Training Score:',rf.score(X_train, y_train))
print('Testing Score:',rf.score(X_test, y_test))
print('Root Mean squared Error:',np.sqrt(rf_mse))

/home/mayday/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training Score: 0.8502631891826451
Testing Score: 0.15913187613348878
Root Mean squared Error: 4.752970720156676


### KNN Regressor

In [170]:
knn = KNeighborsRegressor()
knn.fit(X_train,y_train)
knn_pred = knn.predict(X_test)
knn_mse = metrics.mean_squared_error(y_test,knn_pred)

print('Training Score:',knn.score(X_train, y_train))
print('Testing Score:',knn.score(X_test, y_test))
print('Root Mean squared Error:',np.sqrt(knn_mse))

Training Score: 0.38633035189946263
Testing Score: 0.043057365642581114
Root Mean squared Error: 5.070422467605633


### Ridge Regressor

In [171]:
ridge = Ridge()
ridge.fit(X_train,y_train)
ridge_pred = ridge.predict(X_test)
ridge_mse = metrics.mean_squared_error(y_test,ridge_pred)

print('Training Score:',ridge.score(X_train, y_train))
print('Testing Score:',ridge.score(X_test, y_test))
print('Root Mean squared Error:',np.sqrt(ridge_mse))

Training Score: 0.17864733249751674
Testing Score: 0.17506445068851084
Root Mean squared Error: 4.707726280447419


### SVM Regressor

In [172]:
svc = svm.SVR()
svc.fit(X_train,y_train)
svc_pred = svc.predict(X_test)
svm_mse = metrics.mean_squared_error(y_test,svc_pred)

print('Training Score:',svc.score(X_train, y_train))
print('Testing Score:',svc.score(X_test, y_test))
print('Root Mean squared Error:',np.sqrt(svm_mse))

/home/mayday/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Training Score: 0.21476883693213958
Testing Score: -0.05467729616496686
Root Mean squared Error: 5.323055481972788


In [211]:
models = pd.DataFrame({
    'Model': ['Linear Regressor','Random Forest Regressor','KNN Regressor','Ridge Regressor','SVM Regressor'],
    'Training Score': [ lr.score(X_train, y_train),rf.score(X_train, y_train), knn.score(X_train, y_train), ridge.score(X_train, y_train),svc.score(X_train, y_train)],
    'Test Score': [ lr.score(X_test, y_test),rf.score(X_test, y_test), knn.score(X_test, y_test), ridge.score(X_test, y_test),svc.score(X_test, y_test),],
    'Mean Squared Error': [lr_mse, rf_mse, knn_mse, ridge_mse, svm_mse]
    })
models.sort_values(by='Mean Squared Error')

,Model,Training Score,Test Score,Mean Squared Error
3,Ridge Regressor,0.178647,0.175064,22.162687
0,Linear Regressor,0.179255,0.173413,22.207050
1,Random Forest Regressor,0.850263,0.159132,22.590731
2,KNN Regressor,0.386330,0.043057,25.709184
4,SVM Regressor,0.214769,-0.054677,28.334920


#### Random Forrest is giving maximum Test Score, so using GridSearch to find best hyper-parameters for the same

In [175]:
parameters = [{'max_depth' : [5, 7, 9, 11, 12],
               'max_features' : [2, 3, 4, 5],
               'n_estimators' : [10, 15, 20, 17, 22, 25],
               'random_state' : [7, 8, 10, 12]}]

from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator = rf,param_grid = parameters,
                           scoring = 'neg_mean_squared_error',n_jobs = -1, cv = 10)

grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
print("Best accuracy of the model for the training set is:", best_accuracy)

best_params = grid_search.best_params_
print("Best parameters of the model for the training set is:", best_params)

Best accuracy of the model for the training set is: -18.912471188691182
Best parameters of the model for the training set is: {'max_depth': 9, 'max_features': 3, 'n_estimators': 25, 'random_state': 7}


### Using GridCV Parameters

In [176]:
regressor = RandomForestRegressor(max_depth = 9, max_features = 3,
                                  n_estimators = 25, random_state = 7)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
           max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=25, n_jobs=None, oob_score=False, random_state=7,
           verbose=0, warm_start=False)

### Applying k-fold cross validation again to check mean accuracies in dataset


In [204]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator = regressor, X = X_train, scoring= 'neg_mean_squared_error',
                             y = y_train, cv = 5, n_jobs = -1)

print("Showing all 10 of K-Fold Cross Validation MSE(s):\n", accuracies)
mse_mean = accuracies.mean()
print("\nMean of MSE:\n", mse_mean)


Showing all 10 of K-Fold Cross Validation MSE(s):
 [-19.48103203 -18.93587168 -21.79884212 -20.8833863  -15.36952174]

Mean of MSE:
 -19.29373077421704


## Now training our classifier on full training set

In [183]:
regressor.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
           max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=25, n_jobs=None, oob_score=False, random_state=7,
           verbose=0, warm_start=False)

### According to above findings we use the following classifier for our predictions

In [189]:
test = pd.read_csv('problem_2_test.csv',parse_dates=['memcreateddate', 'mobilepurchasedate',
       'claimintdate','pickupStartdate'])
test.head()

test['mem-purchase'] = (test.mobilepurchasedate - test.memcreateddate).dt.days
test['pickup-claim'] = (test.pickupStartdate - test.claimintdate).dt.days
test['claim-mem'] = (test.claimintdate - test.memcreateddate).dt.days
test['claim-purchase'] = (test.claimintdate - test.mobilepurchasedate).dt.days

In [190]:
test.claimtype = test.claimtype.map(claim_map)
test.mobmake = test.mobmake.map(mob_map)
test.custpincodecategory = test.custpincodecategory.map(pin_map)
test.head()

,srno,mobmake,hubid,hubpincode,pickuppartnerid,deliverypartnerid,insuranceCompanyId,custpincodecategory,claimtype,custpincode,memcreateddate,mobilepurchasedate,claimintdate,servicecntrid,pickupStartdate,mem-purchase,pickup-claim,claim-mem,claim-purchase
0,2829088,0.0,3,560011,233,233,131,0,0,560093,2017-07-30,2017-07-27,2018-02-01,4467,2018-02-06,-3,5,186,189
1,2829866,1.0,2,110018,233,233,228,0,0,110034,2017-07-17,2017-01-25,2018-02-01,4388,2018-02-08,-173,7,199,372
2,2894489,1.0,3,560011,233,233,228,2,0,590014,2018-01-12,2018-01-11,2018-02-21,4460,2018-02-28,-1,7,40,41
3,2876708,1.0,2,110018,233,233,228,1,0,250110,2017-08-07,2017-05-21,2018-02-16,4388,2018-02-26,-78,10,193,271
4,2890283,0.0,22,400104,233,233,131,2,0,496001,2017-11-10,2017-09-22,2018-02-20,4581,2018-02-23,-49,3,102,151


### Checking for null values in test data



In [191]:
print(test.isna().sum())

srno                   0
mobmake                4
hubid                  0
hubpincode             0
pickuppartnerid        0
deliverypartnerid      0
insuranceCompanyId     0
custpincodecategory    0
claimtype              0
custpincode            0
memcreateddate         0
mobilepurchasedate     0
claimintdate           0
servicecntrid          0
pickupStartdate        0
mem-purchase           0
pickup-claim           0
claim-mem              0
claim-purchase         0
dtype: int64


### Filling null values with median value of column



In [192]:
test.fillna(test.mean(),inplace=True)

### Creating a new file for saving the predicted labels for test data



In [193]:
test = test[X.columns]

In [194]:
answers = pd.DataFrame(regressor.predict(test))
answers.to_csv('test_data_answers.csv', index=False,header=['Dataset'])

### Appending to test data and saving file


In [195]:
test['Dataset'] = answers
test.head()
test.to_csv('test_data with answers.csv')